In [10]:
from collections import OrderedDict
import pandas as pd
import pickle

from ml_pee_stego_v3 import MLPEEStego

# Set up the secret data

In [11]:
from utils.data_preparation import get_filenames_from_folder, get_secret_file

# Tentukan folder path tempat secret
SECRET_FOLDER_PATH = 'keys/bin'

# Dapatkan semua nama file .txt dari folder
secret_filenames = get_filenames_from_folder('txt', SECRET_FOLDER_PATH)
secret_filenames.sort(reverse=True)

# Masukkan ke dalam dictionary berisi nama filename beserta isi secret filenya
secret_dicts = OrderedDict((secret_filename, get_secret_file(
    SECRET_FOLDER_PATH + '/' + secret_filename + '.txt')) for secret_filename in secret_filenames)

for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")

Secret untuk secret_0.99_bps
Secret untuk secret_0.66_bps
Secret untuk secret_0.51_bps
Secret untuk secret_0.48_bps
Secret untuk secret_0.39_bps
Secret untuk secret_0.32_bps
Secret untuk secret_0.21_bps
Secret untuk secret_0.15_bps
Secret untuk secret_0.08_bps


# Set up the model

In [12]:
import warnings
from sklearn.exceptions import InconsistentVersionWarning

# Menonaktifkan peringatan InconsistentVersionWarning
warnings.simplefilter("ignore", category=InconsistentVersionWarning)

models = OrderedDict()

# models['SVR'] = 'models/svr_model.pkl'
models['SVR 2.0'] = 'models/svr_model_2.0.pkl'
models['LASSO'] = 'models/lasso_model.pkl'
models['ElasticNet'] = 'models/elastic_net_model.pkl'
models['Bayesian Ridge'] = 'models/bayesian_ridge_model.pkl'
models['SGD Regressor'] = 'models/sgd_model.pkl'

# Set up patient

In [13]:
from utils.data_preparation import slice_batch_data

# Tentukan folder path tempat secret
patient_folder_path = 'data/mit-bih-arrhythmia-database-1.0.0/'

# Dapatkan semua nama file .dat dari folder
patient_codes = get_filenames_from_folder('dat', patient_folder_path)
patient_codes.sort()

signals = []
for patient_code in patient_codes:
    batch_signals = slice_batch_data(patient_code=patient_code, max_batch=1)
    signals.extend(batch_signals)

print(len(signals))

46


# Run model test

In [16]:
from utils.telegram_bot import send_message
from datetime import datetime

right_now = datetime.now()
test_result = []

try:
    for model_name, model_path in models.items():
        print(f"Model {model_name}")
        with open(models.get(model_name), 'rb') as model_file:
            model = pickle.load(model_file)

        stego = MLPEEStego(model)

        for payload_rate in [1, 2, 3]:
            for threshold in [0, 1]:
                for index_signal, original_signal in enumerate(signals):
                    for secret_name, secret_data in secret_dicts.items():
                        print(f"Pada model {model_name} payload_rate {payload_rate} threshold {threshold} signals {index_signal} secret {secret_name}")
                        print(">>Embedding...")
                        watermarked_signal, mirror_data, last_phase, last_i, last_embedded_bit_total, result = stego.embed(
                            original_signal, secret_data, payload_rate=payload_rate, threshold=threshold)
                        print(f"Last Phase: {last_phase}, Last I: {last_i}, Last embedded bit: {last_embedded_bit_total}")
                        print(">>Extracting...")
                        extracted_original_signal, extracted_secret_data = stego.extract(
                            watermarked_signal, mirror_data, last_phase, last_i, last_embedded_bit_total, payload_rate=payload_rate, threshold=threshold)

                        print(">>", original_signal[0:20], secret_data[0:20])
                        print(">>", extracted_original_signal[0:20],
                            extracted_secret_data[0:20])
                        print(">>", "Len -> Ori:", len(original_signal), ", Ex-ori:", len(extracted_original_signal),
                            ", Sec:", len(
                                    secret_data), ", Ex-sec:", len(extracted_secret_data))
                        print("******\n******")

                        test_result.append([model_name, payload_rate, threshold, index_signal, secret_name, len(secret_data), len(extracted_secret_data), result.ncc, result.prd, result.snr, result.timer])
except Exception as err:
    print(repr(err))
    send_message(f"Terjadi masalah {right_now}")


IndexError('list index out of range')


In [ ]:
out_csv = f"result_{right_now.strftime('%Y%m%d_%H%M%S.csv')}"

In [ ]:
df = pd.DataFrame(test_result, columns=['model_name', 'payload_rate', 'threshold', 'index_signal', 'secret_name', 'len_secret_data', 'len_extracted_secret_data', 'ncc', 'prd', 'snr', 'time'])
df.to_csv(f'out/{out_csv}', index=False)